## What to Expect From This Notebook
- This notebook is a demonstration of the proper way to calculate the number of pixels to project a satellite image to when given latitude and longitude or path and row WRS-2 coordinates
- This notebook takes into account the geoid shape Earth using the latest World Geodetic System (WGS84)

## Algorithmic Steps
- Turn path and row WRS coordinates into latitude and longitude coordinates if not already
- Determine the number of meters per degree of latitude and longitude
- Determine the pixel count of one degree of latitude and longitude using the meters per degree and satellite image resolution
- Calculate the tile resolution that should be used using meters per degree and satellite image resolution
- Determine tile size in pixels using tile resolution and meters per degree of latitude and longitude
***
## How It Works
To determine the appropriate pixel size to use for a geographic region there are three basic factors to take into account:
  - <font color=green>The resolution of the satellite</font>
    - Landsat satellites have a resolution of 30m per pixel
  - <font color=green>The latitude</font>
    - The longitude does not matter.  The meters per degree of longitude are determined by the latitude since the longitudinal lines converge at the poles and reach their maxima at the equator. 
  - <font color=green>The actual shape of the Earth</font>
    - Earth is not a true sphere, it is actually a geoid.  Becuase of this we have to use a gravimetrically oriented ellipsoid to approximate Earth.
    - We will use the WGS84 measurements of the Earth's semimajor and semiminor axes since that is what GPS relies on and is very accurate.  It is estimated that the true center of Earth's mass lies within 2cm of the WGS84 measurements.
![](diagrams/ingestion/WGS_84.png)  

In [ ]:
import math
import pandas as pd
import requests
import time

#Getter for Epoch time
current_milli_time = lambda: int(round(time.time() * 1000))

## What are WRS-2 Path and Rows?

USGS satellites follow tilted, nearly vertical paths.  These paths are geodesic, one full cycle does not end where it starts.
- <font color=purple>Paths</font> are the intuitive name given to the vertical rows of the geodesic paths taken by the satellites
- <font color=purple>Rows</font> are slightly more abstract in that they are 
![](diagrams/ingestion/geodesic_on_an_oblate_ellipsoid.png)  
![](diagrams/ingestion/wrs2.gif)  

In [ ]:
#For consistency and simplicity, since the alternative is retreiving and parsing a shapefile
#we will ping USGS' web tool for path row conversion
def pr_to_ll(path, row, show_url = False):
    """Path/Row WRS coordinates to Latitude and Longitude.
    This function uses USGS' landsat calculator for lat/long conversions via an HTTP request
    """
    
    url = "https://landsat.usgs.gov/landsat/lat_long_converter/tools_latlong.php?rs=convert_pr_to_ll"
    a = [path,row]    
    for i in range(len(a)):
        url = url + "&rsargs[]=" + str(a[i])
    url = url + "&rsrnd=" + str(current_milli_time())
    if(show_url): print("view directly from USGS:", url)
    r = requests.get(url)  
    table = pd.read_html(r.text)[0]
    lat, lon = table[1][1], table[3][1]
    return {"Latitude":float(lat),"Longitude":float(lon)}

In [1]:
#This is the result of using the geodetic distance formula using the
#ellipsoid measurements from WGS84 reduced to constants.
#it should be accurate to within ~1 meter
def meters_per_degree(degrees_latitude):
    """Gets meters per degree of latitude and longitude from latitude"""
    lat_rads = math.radians(degrees_latitude)
    mpd_lat = 111132.92 - 559.82*math.cos(2*lat_rads)+1.175*math.cos(4*lat_rads)
    mpd_lon = 111412.84 * math.cos(lat_rads) - 93.5 * math.cos(3*lat_rads)
    return {"lat_mpd":mpd_lat, "lon_mpd":mpd_lon}

In [4]:
def pixel_counts(latitude_position, image_resolution = 30):
    """Pixel count information based on the latitude and image resolution.
    This function returns a dictionary of values indicating the pixel counts, tile-sizes,
    resolutions, and estimated meters per degrees for both latitude and longitude.
    """
    
    temp = meters_per_degree(latitude_position)
    lat_pxl_count = math.floor(temp["lat_mpd"]/image_resolution)
    lon_pxl_count = math.floor(temp["lon_mpd"]/image_resolution)
    res_lat = -(image_resolution/temp["lat_mpd"])
    res_lon = image_resolution/temp["lon_mpd"]
    tile_size_lat = -(res_lat*lat_pxl_count)
    tile_size_lon = res_lon*lon_pxl_count
    cond1 = (tile_size_lat / res_lat)%1==0
    cond2 = (tile_size_lon / res_lon)%1==0
    if(cond1 & cond2):
        return {"lat_pxl_count":lat_pxl_count, "lon_pxl_count":lon_pxl_count,
                "resolution_lat":res_lat, "resolution_lon":res_lon, "tile_size_lat":tile_size_lat,
                "tile_size_lon":tile_size_lon, "meters_per_deg_lat":temp["lat_mpd"], "meters_per_deg_lon":temp["lon_mpd"]}
    

In [5]:
def pr_to_pixel_counts(path = 0, row = 0, resolution = 30):
    """Path/Row WRS coordinates to pixel count information.
    This function is a wrapper for the pixel_counts function that allows you to pass
    path and row coordinates in order to get the pixel counts, tile-sizes,
    resolutions, and estimated meters per degrees for both latitude and longitude.
    """
    
    lat_lon = pr_to_ll(path,row)
    return {**lat_lon, **pixel_counts(lat_lon["Latitude"])}

def ll_to_pixel_counts(latitude = 0, longitude = 0, resolution = 30):
    """Latitude and Longitude coordinates to pixel count information.
    This function is a wrapper for the pixel_counts function that allows you to pass
    latitude and longitude coordinates in order to get the pixel counts, tile-sizes,
    resolutions, and estimated meters per degrees for the given latitude and longitude.
    """
    
    lat_lon = {"Latitude":latitude,"Longitude":longitude}
    return {**lat_lon, **pixel_counts(lat_lon["Latitude"])}

In [6]:
#in order to get the meters per degree for latitude and longitude directly, use:
meters_per_degree(77.049)

{'lat_mpd': 111637.22846518604, 'lon_mpd': 25028.242043474733}

In [7]:
#to get all of the pixel count information from latitude/Longitude format:
ll_to_pixel_counts(77.049,-25.208,30) #only latitude and image resolution matter for the calculations

({'Latitude': 77.049, 'Longitude': -25.208},
 {'lat_pxl_count': 3721,
  'lon_pxl_count': 834,
  'meters_per_deg_lat': 111637.22846518604,
  'meters_per_deg_lon': 25028.242043474733,
  'resolution_lat': -0.0002687275599049422,
  'resolution_lon': 0.001198645911602149,
  'tile_size_lat': 0.9999352504062899,
  'tile_size_lon': 0.9996706902761923})

In [8]:
#in order to get pixel count information from path/row format:
pr_to_pixel_counts(4,5,30) #with path, row, and image_resolution as the arguments

({'Latitude': 77.049, 'Longitude': -25.208},
 {'lat_pxl_count': 3721,
  'lon_pxl_count': 834,
  'meters_per_deg_lat': 111637.22846518604,
  'meters_per_deg_lon': 25028.242043474733,
  'resolution_lat': -0.0002687275599049422,
  'resolution_lon': 0.001198645911602149,
  'tile_size_lat': 0.9999352504062899,
  'tile_size_lon': 0.9996706902761923})

In [9]:
meters_per_degree(35)

{'lat_mpd': 110940.55018114275, 'lon_mpd': 91288.25522675944}

In [ ]:
meters_per_degree_2(35)

In [ ]:
2**3

In [12]:
dicA = {"A":1, "B":3}
dicB = {"B":2 ,"C":3}
dicC = {**dicA,**dicB}
print(dicC)

{'C': 3, 'A': 1, 'B': 2}
